In [268]:
import pandas as pd
import numpy as np
import warnings 

warnings.filterwarnings("ignore")

In [269]:
from pulp import *

In [315]:
user_scores =  pd.read_csv("Job_scoring_users.csv")
user_preferences = pd.read_csv("user_preferences.csv")

In [316]:
users = list(user_scores.Candidate.values)
jobs = list( user_scores.columns[1:user_scores.shape[1]].values)
user_scores_array = user_scores.iloc[:,range(1,user_scores.shape[1])].values

In [318]:
print( "  Maximum user score is : " ,user_scores_array.max())

  Maximum user score is :  58


In [319]:
max_score = 100

In [320]:
user_preferences = user_preferences.replace({
    1:5 * max_score, 
    2:4 * max_score,
    3:3 * max_score,
    4:2 * max_score,
    5:1 * max_score})
user_preferences_num_cols =  user_preferences.iloc[:,range(1,user_scores.shape[1])]

In [292]:
#user_preferences = user_preferences.replace({1:1,2:0.1,3:0.01,4:0.001,5:0.0001})*10000
#user_preferences_num_cols =  user_preferences.iloc[:,range(1,user_scores.shape[1])]

In [321]:
user_preferences_0_array =  user_preferences_num_cols.values

### Decision variable: 

In [308]:
prob = LpProblem("Matching Jobs", LpMaximize)
y = LpVariable.dicts("pair", [(i,j)  for i in range(len(users)) for j in range(len(jobs)) ] ,cat='Binary')

### Maximization problem : 

In [323]:
prob += lpSum([ (user_preferences_0_array[i][j] + user_scores_array[i][j]) * y[(i,j)] for i in range(len(users)) for j in range(len(jobs)) ])

### Contraints:

In [324]:
## each person should be given only one job 

for i in range(len(users)):
    prob += lpSum(y[(i,j)] for j in range(len(jobs))) <= 1

## place constraints: 
for j in [0]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 4
                  
for j in [1]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     
                  
for j in [2]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     
                  
for j in [3]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     

for j in [4]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 3     
    
prob.solve()

1

### Creating a record of all the score values :

In [325]:
scores_array = user_preferences_0_array.copy()
scores_array[:,:] = 0

for i in range(len(users)):
    for j in range(len(jobs)):
        scores_array[i,j] = user_preferences_0_array[i][j] + user_scores_array[i][j] 

## Presenting results with the cells highlighted 

##### Creating array of matched values: 

In [327]:
matches_array =  user_preferences_0_array.copy()
matches_array[:,:] = 0

for i in range(len(users)):
    for j in range(len(jobs)):
        matches_array[i,j] =  y[(i,j)].varValue 
        
matches_df =  pd.DataFrame(matches_array, columns =  user_scores.columns[range(1,user_scores.shape[1] )] )
matches_df['Candidate'] = user_scores["Candidate"]
matches_df =matches_df[user_scores.columns]
matches_df

### Combining users preferences and scores :

In [365]:
results_df =  user_scores.copy()
for colnames  in results_df.columns[range(1,len(results_df.columns))]:
    results_df[colnames] = user_scores[colnames].astype('str') + "," + user_preferences[colnames].astype('str')

### Results with coloured tiles : 

In [366]:

style_df = (
        matches_df == 1                  # Compare DataFrames
).replace({
    True: 'background-color:yellow',  # True Styles
    False: ''                      # False Styles
})

results_df =  results_df.style.apply(lambda _: style_df, axis=None)
results_df

,Candidate,Panchkula,Ambala,Faridabad,Gurgaon,Panipat
0,A,"52,500","42,400","12,100","22,200","32,300"
1,B,"44,400","54,500","14,100","24,200","34,300"
2,C,"45,400","35,300","55,500","15,100","25,200"
3,D,"42,400","32,300","12,100","52,500","22,200"
4,E,"56,500","46,400","16,100","26,200","36,300"
5,F,"49,500","49,500","29,200","39,300","29,200"
6,G,"44,400","34,300","14,100","24,200","54,500"
7,H,"48,400","58,500","18,100","28,200","38,300"
8,I,"54,500","44,400","14,100","24,200","34,300"
9,J,"53,500","43,400","13,100","23,200","33,300"


### Caste constraints : 

Introducing A, F, J  as SC:  

In [370]:
user_SC_index =  [0,5,9]

Setting 2 as number of SC required : 

In [372]:
SC_required =  2

### Decision variable: 

In [374]:
prob = LpProblem("Matching Jobs", LpMaximize)
y = LpVariable.dicts("pair", [(i,j)  for i in range(len(users)) for j in range(len(jobs)) ] ,cat='Binary')

### Maximization problem : 

In [375]:
prob += lpSum([ (user_preferences_0_array[i][j] + user_scores_array[i][j]) * y[(i,j)] for i in range(len(users)) for j in range(len(jobs)) ])

### Contraints:

In [376]:
## each person should be given only one job 

for i in range(len(users)):
    prob += lpSum(y[(i,j)] for j in range(len(jobs))) <= 1

## place constraints: 
for j in [0]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 4
                  
for j in [1]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     
                  
for j in [2]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     
                  
for j in [3]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 2     

for j in [4]:
    prob += lpSum(y[(i,j)] for i in range(len(users))) == 3    
    
    
## SC constraints: 

prob += lpSum([  y[(i,j)] for i in user_SC_index for j in range(len(jobs))  ]) == SC_required
    
    
prob.solve()

1

In [378]:
matches_array =  user_preferences_0_array.copy()
matches_array[:,:] = 0

for i in range(len(users)):
    for j in range(len(jobs)):
        matches_array[i,j] =  y[(i,j)].varValue 
        
matches_df =  pd.DataFrame(matches_array, columns =  user_scores.columns[range(1,user_scores.shape[1] )] )
matches_df['Candidate'] = user_scores["Candidate"]
matches_df =matches_df[user_scores.columns]
matches_df

results_df =  user_scores.copy()
for colnames  in results_df.columns[range(1,len(results_df.columns))]:
    results_df[colnames] = user_scores[colnames].astype('str') + "," + user_preferences[colnames].astype('str')

    
style_df = (
        matches_df == 1                  # Compare DataFrames
).replace({
    True: 'background-color:yellow',  # True Styles
    False: ''                      # False Styles
})    
    

results_df =  results_df.style.apply(lambda _: style_df, axis=None)
results_df

,Candidate,Panchkula,Ambala,Faridabad,Gurgaon,Panipat
0,A,"52,500","42,400","12,100","22,200","32,300"
1,B,"44,400","54,500","14,100","24,200","34,300"
2,C,"45,400","35,300","55,500","15,100","25,200"
3,D,"42,400","32,300","12,100","52,500","22,200"
4,E,"56,500","46,400","16,100","26,200","36,300"
5,F,"49,500","49,500","29,200","39,300","29,200"
6,G,"44,400","34,300","14,100","24,200","54,500"
7,H,"48,400","58,500","18,100","28,200","38,300"
8,I,"54,500","44,400","14,100","24,200","34,300"
9,J,"53,500","43,400","13,100","23,200","33,300"
